In [2]:
import json
import numpy as np


def calculate_polys(data: list[dict], x_label: str, y_label: str):
    freqs: list[str] = {p['freq'] for p in data}
    polys: dict[dict] = {}

    for freq in freqs:
        points = [p for p in data if p['freq'] == freq]

        x = [0, *[p[x_label] for p in points]]
        y = [0, *[p[y_label] for p in points]]
        # x = [p[x_label] for p in points]
        # y = [p[y_label] for p in points]

        power = 2
        temp = np.poly1d(np.polyfit(x, y, power))

        alpha = 'abcdefghijklmnopqrstuvwxyz'
        poly = {}
        for i, p in enumerate(range(power, -1, -1)):
            poly[alpha[i]] = round(temp[p], 10)

        polys[freq] = poly

    return polys


def create_poly_header(polys):
    header = []

    # make sure the outputs are in order
    bands = [p for p in polys['fwd']]
    bands.sort()

    header.append('polynomial_t forwardCalibrationTable[NUM_OF_BANDS] = {\n')
    for band in bands:
        header.append('    {')

        poly: dict = polys['fwd'][band]
        out = []
        for value in poly.values():
            out.append(str(value))
        header.append(', '.join(out))

        header.append('},')
        header.append(' // ' + band + '\n')
    header.append('};\n\n')

    header.append('polynomial_t reverseCalibrationTable[NUM_OF_BANDS] = {\n')
    for band in bands:
        header.append('    {')

        poly: dict = polys['rev'][band]
        out = []
        for value in poly.values():
            out.append(str(value))
        header.append(', '.join(out))

        header.append('},')
        header.append(' // ' + band + '\n')
    header.append('};\n\n')

    return ''.join(header)


with open('fwd.json', 'r') as f:
    fwd = json.load(f)['data']

with open('fwd_attenuated.json', 'r') as f:
    fwd.extend(json.load(f)['data'])

with open('rev.json', 'r') as f:
    rev = json.load(f)['data']

polys = {
    'fwd': calculate_polys(fwd, 't_fwd_volts', 'm_fwd'),
    'rev': calculate_polys(rev, 't_rev_volts', 'm_fwd'),
}


header = create_poly_header(polys)

print(header)

polynomial_t forwardCalibrationTable[NUM_OF_BANDS] = {
    {3.4479e-05, 0.012242772, -0.5888886198}, // 01800000
    {2.40674e-05, 0.0144357467, -1.0384081397}, // 03500000
    {2.32939e-05, 0.0121373473, -0.6636951614}, // 07000000
    {2.46085e-05, 0.0107465893, -0.3731147554}, // 10100000
    {2.36828e-05, 0.0145613211, -0.9381904235}, // 14000000
    {2.48176e-05, 0.0134516666, -0.7756303431}, // 18068000
    {2.43053e-05, 0.0138282772, -0.87842567}, // 21000000
    {2.44645e-05, 0.0146616079, -1.0958326768}, // 24890000
    {2.42895e-05, 0.013256318, -0.6386329677}, // 28000000
    {2.59059e-05, 0.0145094203, -0.6225379768}, // 50000000
};

polynomial_t reverseCalibrationTable[NUM_OF_BANDS] = {
    {7.422e-06, 0.0043297273, 0.0418036189}, // 01800000
    {5.711e-06, 0.0039180843, 0.0347077466}, // 03500000
    {5.463e-06, 0.0038557671, 0.0336771535}, // 07000000
    {5.7766e-06, 0.0039250007, 0.0378529411}, // 10100000
    {5.9187e-06, 0.0039376254, 0.0422536579}, // 14000000
    